# POSの模擬データ解析


## POS (Point of Sale) モックデータの生成

次のコードは、スーパーマーケットやコンビニなどでの商品購入情報を模倣するためのPOSデータを生成します。

### ステップごとの説明

1. **必要なライブラリのインポート**:
    ```python
    import pandas as pd
    import numpy as np
    ```
    `pandas` はデータ処理に、`numpy` は数値計算に使用します。

2. **基本設定**:
    - `n_products`: 商品の数。500商品を想定しています。
    - `n_receipts`: レシートの数。1500のレシートを想定しています。
    - `product_ids` と `receipt_ids`: 商品とレシートのIDを生成します。

3. **数量の決定**:
    `get_quantity` 関数は、購入する商品の数量をランダムに決定します。数量は1〜5の間で、それぞれの数量が選ばれる確率が設定されています。

4. **データフレームの初期化**:
    各レシートと商品に対する数量を0で初期化します。

5. **データの生成**:
    以下のパターンで商品の購入情報を生成します:
    - **p1, p2, p3の購入レシート**: このパターンのレシートは、全レシートの30%を占めます。
    - **p4, p5, p6, p7の購入レシート**: このパターンのレシートは、全レシートの次の20%を占めます。
    - **p1, p7の購入レシート**: このパターンのレシートは、全レシートの次の10%を占めます。
    - **p12のみの購入レシート**: このパターンのレシートは、全レシートの次の5%を占めます。
    - **ランダムな商品の購入レシート**: 残りのレシートで、ランダムな商品が購入されます。

6. **ラベルの割り当て**:
    `assign_labels` 関数は、各レシートの購入パターンに基づいてラベルを割り当てます。このコードでは、4つの特定の購入パターンとそれ以外の購入パターンにラベルを割り当てています。

最終的に、このコードは各レシートに購入された商品の数量を示すデータフレームを出力します。


In [ ]:
import pandas as pd
import numpy as np

# 設定
n_products = 500
n_receipts = 1500
product_ids = ["p" + str(i) for i in range(1, n_products + 1)]
receipt_ids = ["r" + str(i) for i in range(1, n_receipts + 1)]

# 1〜5の範囲の数量を取得
def get_quantity():
    p = [0.50, 0.2475, 0.165, 0.0825, 0.005]
    return np.random.choice([1,2,3,4,5], p=p)


# データフレームの初期化
df = pd.DataFrame(0, index=receipt_ids, columns=product_ids)

# p1, p2, p3の購入レシート
for i in range(int(n_receipts * 0.30)):
    df.loc[receipt_ids[i], ["p1", "p2", "p3"]] = [get_quantity() for _ in range(3)]
    random_products = np.random.choice(product_ids[3:], np.random.randint(0, 3), replace=False)
    for product in random_products:
        df.loc[receipt_ids[i], product] = get_quantity()

# p4, p5, p6, p7の購入レシート
for i in range(int(n_receipts * 0.30), int(n_receipts * 0.50)):
    df.loc[receipt_ids[i], ["p4", "p5", "p6", "p7"]] = [get_quantity() for _ in range(4)]
    random_products = np.random.choice(product_ids[7:], np.random.randint(0, 3), replace=False)
    for product in random_products:
        df.loc[receipt_ids[i], product] = get_quantity()

# p1, p7の購入レシート
for i in range(int(n_receipts * 0.50), int(n_receipts * 0.60)):
    df.loc[receipt_ids[i], ["p1", "p7"]] = [get_quantity() for _ in range(2)]
    excluded_products = ["p1", "p7"]
    random_products = np.random.choice(product_ids[7:], np.random.randint(0, 3), replace=False)
    for product in random_products:
        df.loc[receipt_ids[i], product] = get_quantity()
    
# p12のみの購入レシート
for i in range(int(n_receipts * 0.60), int(n_receipts * 0.65)):
    df.loc[receipt_ids[i], "p12"] = get_quantity()

# ランダムな商品を購入するレシート
for i in range(int(n_receipts * 0.65), n_receipts):
    random_products = np.random.choice(product_ids, np.random.randint(5, 11), replace=False)
    for product in random_products:
        df.loc[receipt_ids[i], product] = get_quantity()

def assign_labels(df):
    labels = []

    for index, row in df.iterrows():
        if row['p1'] > 0 and row['p2'] > 0 and row['p3'] > 0:
            labels.append('A')
        elif row['p4'] > 0 and row['p5'] > 0 and row['p6'] > 0 and row['p7'] > 0:
            labels.append('B')
        elif row['p1'] > 0 and row['p7'] > 0:
            labels.append('C')
        elif row['p12'] > 0 and row.sum() == row['p12']:
            labels.append('D')
        else:
            labels.append('Other')

    return labels
        
labels = assign_labels(df)

df


## 高次元解析

In [ ]:
# プロット関数定義
import matplotlib.pyplot as plt

def plot_reduction_results_with_labels(results, labels, titles):
    # resultsやtitlesがリストでない場合、リストに変換
    if not isinstance(results, list):
        results = [results]
    if not isinstance(titles, list):
        titles = [titles]
    
    fig, axes = plt.subplots(1, len(results), figsize=(10 * len(results), 5))
    
    # resultsが1つの場合、axesをリストに変換
    if len(results) == 1:
        axes = [axes]
    
    unique_labels = sorted(list(set(labels)))
    for ax, result, title in zip(axes, results, titles):
        for i, label in enumerate(unique_labels):
            idx = [index for index, x in enumerate(labels) if x == label]
            ax.scatter(result[idx, 0], result[idx, 1], s=10, label=label, c=label_to_color[label])
        ax.set_title(title)
        ax.axis('equal')
        ax.set_xlim([min(result[:, 0]), max(result[:, 0])])
        ax.set_ylim([min(result[:, 1]), max(result[:, 1])])
        ax.legend()
    plt.tight_layout()
    plt.show()
    
# RGBとCMYを使用して色を区別
label_to_color = {
    'A': 'b',
    'B': 'g',
    'C': 'orange', 
    'D': 'r',
    'Other': 'm'
}

### PCA, MDSによる解析

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import MDS

pca_result = PCA(n_components=2).fit_transform(df.values)
mds_result = MDS(n_components=2).fit_transform(df.values)

plot_reduction_results_with_labels([pca_result, mds_result], labels, 
                                  ["PCA", "MDS"])

### t-SNE, UMAPによる解析

In [ ]:
from sklearn.manifold import TSNE
import umap

# 各手法で次元削減
tsne_result = TSNE(n_components=2).fit_transform(df.values)
umap_result = umap.UMAP(n_components=2).fit_transform(df.values)

plot_reduction_results_with_labels([tsne_result, umap_result], labels, 
                                  ["t-SNE", "UMAP"])

### toorPIAによる解析

In [ ]:
import toorpia.utils as toorpia

# 各手法で次元削減
toorpia_result = toorpia.fit_transform(df.values, item_normalization=False, vector_normalization=False)

plot_reduction_results_with_labels([toorpia_result], labels, 
                                  ["toorPIA"])

In [ ]:
# toorPIA
import os
import toorpia.utils as toorpia

working_dir = 'toorpia_analysis'
os.makedirs(working_dir, exist_ok=True)

df['labels'] = labels
rawdata = os.path.join(working_dir, 'rawdata.csv')
df.to_csv(rawdata, index=False, header=True)
df = df.drop('labels', axis=1)

params = {
    'working_dir': working_dir,
    'rawdata': rawdata,
    'item_normalization': False,
    'vector_normalization': False,
    'type_weight': working_dir + "/type_weight.csv",
}
toorpia.create_type_weight(params, force=True)
x, y = toorpia.create_basemap(params, force=True)

## POSデータを転置にして解析をしてみる

In [ ]:
# データフレームを転置
df_transposed = df.transpose()
df_transposed

In [ ]:
# toorPIA
import os
import toorpia.utils as toorpia

working_dir = 'toorpia_analysis_transposed'
os.makedirs(working_dir, exist_ok=True)

rawdata = os.path.join(working_dir, 'rawdata.csv')
df_transposed.to_csv(rawdata, index=False, header=True)
df_transposed['label'] = df_transposed.index

params = {
    'working_dir': working_dir,
    'rawdata': rawdata,
    'item_normalization': False,
    'type_weight': working_dir + "/type_weight.csv",
}
toorpia.create_type_weight(params, force=True)
x, y = toorpia.create_basemap(params, force=True)

POSの転置データから何がわかるか各自考察してください。